In [1]:
!pip3 install minio
!pip3 install pandas
!sudo apt-get install -y mosquitto-clients

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mosquitto-clients is already the newest version (1.6.9-1).
0 upgraded, 0 newly installed, 0 to remove and 176 not upgraded.


In [2]:
import sys
import  datetime
import os
from minio import Minio
import csv
import json
import pandas as pd
import time
import subprocess
from subprocess import Popen, PIPE
import requests
from IPython.display import display

In [3]:
#df_metrics = pd.DataFrame(columns=['Workload_size','Workload TPT','Average Workload TPT','Average FET','Average CT'])

In [4]:
scenario="WITHOUT_CONF"
user=10
minio_host="172.17.141.197:9001"
bucket_name="aeneas-output"
PROMETHEUS = 'http://172.17.141.197:9096/'


In [5]:
cmd = "./mqtt_client.sh "+str(user)
with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
    output1 = process.communicate()[0].decode("utf-8")
time.sleep(300)

In [6]:
df= pd.read_csv('mqtt.csv')
print(df)

   user                   intime
0     1  2022-08-31 14:03:59.939
1     2  2022-08-31 14:03:59.945
2     3  2022-08-31 14:03:59.951
3     4  2022-08-31 14:03:59.958
4     5  2022-08-31 14:03:59.965
5     6  2022-08-31 14:03:59.972
6     7  2022-08-31 14:03:59.984
7     8  2022-08-31 14:03:59.992
8     9  2022-08-31 14:04:00.000
9    10  2022-08-31 14:04:00.011


In [7]:
#Minio Client
client = Minio(minio_host, access_key = "minio", secret_key ="minio123",secure=False)
# List objects from the bicket and notedown time  stamp  when they stored
objects = client.list_objects(bucket_name,recursive=True)
outtime = []
for obj in objects:
    outtime.append(obj.last_modified)
    client.remove_object(bucket_name, obj.object_name)
print(outtime)

[datetime.datetime(2022, 8, 31, 14, 4, 25, 255000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 20, 278000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 15, 564000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 55, 778000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 5, 258000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 50, 299000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 45, 53000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 36, 209000, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 10, tzinfo=datetime.timezone.utc), datetime.datetime(2022, 8, 31, 14, 4, 30, 385000, tzinfo=datetime.timezone.utc)]


In [8]:
df['outtime']= outtime
df['outtime']= pd.to_datetime(df['outtime']).dt.tz_convert(None)
df['intime']= pd.to_datetime(df['intime'])
df =df.sort_values(by="outtime")
display(df)

,user,intime,outtime
4,5,2022-08-31 14:03:59.965,2022-08-31 14:04:05.258
8,9,2022-08-31 14:04:00.000,2022-08-31 14:04:10.000
2,3,2022-08-31 14:03:59.951,2022-08-31 14:04:15.564
1,2,2022-08-31 14:03:59.945,2022-08-31 14:04:20.278
0,1,2022-08-31 14:03:59.939,2022-08-31 14:04:25.255
9,10,2022-08-31 14:04:00.011,2022-08-31 14:04:30.385
7,8,2022-08-31 14:03:59.992,2022-08-31 14:04:36.209
6,7,2022-08-31 14:03:59.984,2022-08-31 14:04:45.053
5,6,2022-08-31 14:03:59.972,2022-08-31 14:04:50.299
3,4,2022-08-31 14:03:59.958,2022-08-31 14:04:55.778


In [9]:
def get_function_execution_time(cmd):
    with Popen(cmd, stdout=PIPE, stderr=None, shell=True) as process:
        return (process.communicate()[0].decode("utf-8"))    


In [10]:
#cmd1 = "sudo kubectl logs gateway-7b8d9dbb5b-rhnwl  -n openfaas -c gateway | grep getobject | cut -c 71-77 | tail -"
cmd1="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep scaling-aeneas-mqtt | cut -c 93-97 | tail -"+str(len(outtime))
cmd2="ssh ubuntu@172.17.141.197 "+"sudo kubectl logs gateway-7b8d9dbb5b-ggwl6  -n openfaas -c gateway  | grep scaling-aeneas-tocloud | cut -c 96-101 | tail -"+str(len(outtime))
df['aeneas']=(get_function_execution_time(cmd1)).split("\n")[:-1]
df['tocloud']=(get_function_execution_time(cmd2)).split("\n")[:-1]


In [11]:
#TPT: Total Processing Time
#FET: Function Execution Time
#CT: Communication Time
df['TPT']=(df['outtime']-df['intime']).dt.seconds
df['FET'] = (df['aeneas']).astype(float) + (df['tocloud']).astype(float)
df['CT']= (df['TPT']).astype(float) - df['FET']
df =df.sort_values(by="outtime")
display(df)
df.to_csv(str(user)+"_"+scenario+".csv")

,user,intime,outtime,aeneas,tocloud,TPT,FET,CT
4,5,2022-08-31 14:03:59.965,2022-08-31 14:04:05.258,4.546,0.0468,5,4.5928,0.4072
8,9,2022-08-31 14:04:00.000,2022-08-31 14:04:10.000,4.680,0.0453,10,4.7253,5.2747
2,3,2022-08-31 14:03:59.951,2022-08-31 14:04:15.564,4.547,0.0402,15,4.5872,10.4128
1,2,2022-08-31 14:03:59.945,2022-08-31 14:04:20.278,4.597,0.0434,20,4.6404,15.3596
0,1,2022-08-31 14:03:59.939,2022-08-31 14:04:25.255,4.727,0.0364,25,4.7634,20.2366
9,10,2022-08-31 14:04:00.011,2022-08-31 14:04:30.385,4.690,0.0391,30,4.7291,25.2709
7,8,2022-08-31 14:03:59.992,2022-08-31 14:04:36.209,4.796,0.0356,36,4.8316,31.1684
6,7,2022-08-31 14:03:59.984,2022-08-31 14:04:45.053,4.544,0.0397,45,4.5837,40.4163
5,6,2022-08-31 14:03:59.972,2022-08-31 14:04:50.299,4.691,0.0397,50,4.7307,45.2693
3,4,2022-08-31 14:03:59.958,2022-08-31 14:04:55.778,4.648,0.0418,55,4.6898,50.3102


In [12]:
metrics = {'workload_type':str(user)+"_"+scenario,'workload_size':user,'Workload TPT':df['TPT'].sum(),'Average Workload TPT':df['TPT'].mean(),'Average FET':df['FET'].mean(),'Average CT':df['CT'].mean()}
print(metrics)

{'workload_type': '10_WITHOUT_CONF', 'workload_size': 10, 'Workload TPT': 291, 'Average Workload TPT': 29.1, 'Average FET': 4.6874, 'Average CT': 24.412599999999998}


In [13]:
pre_url = PROMETHEUS + '/api/v1/query?query='
timestamp= pd.to_datetime(df['outtime'].iloc[-1], format='%Y-%m-%d %H:%M:%S')
time = (time.mktime(timestamp.timetuple()))
interval= int(df['TPT'].sum()) if int(df['TPT'].sum()) > 60  else 60


In [14]:
def getdataprometheus(url):
    headers= {"Accept": "application/json"}
    res = json.loads(requests.post(url=url, headers=headers).content.decode('utf8', 'ignore'))
    data=res.get('data').get('result')[0].get('value')[1]
    return data

In [15]:
expr_cpu = '100 - (avg (irate(node_cpu_seconds_total{mode="idle"}['+str(interval)+'s])) * 100)&time='+str(time)
expr_memory = '100 - ((sum(node_memory_MemAvailable_bytes) / sum(node_memory_MemTotal_bytes))*100)&time='+str(time)
expr_disk_read = 'sum(rate(node_disk_read_bytes_total['+str(interval)+'s]))&time='+str(time)
expr_disk_write =  'sum(rate(node_disk_written_bytes_total['+str(interval)+'s]))&time='+str(time)
expr_network_transmit= 'sum(rate(node_network_transmit_errs_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
expr_network_recieve='sum(rate(node_network_receive_packets_total{device!~"lo | veth. | docker.* | flannel.* | cali.* | cbr."}['+str(interval)+'s]))&time='+str(time)
metrics['C_CPU'] = getdataprometheus(pre_url+expr_cpu)
metrics['C_RAM'] = getdataprometheus(pre_url+expr_memory)
metrics['C_disk_read']=getdataprometheus(pre_url+expr_disk_read)
metrics['C_disk_write']=getdataprometheus(pre_url+expr_disk_write)
metrics['C_network_recieve']=getdataprometheus(pre_url+expr_network_transmit)
metrics['C_network_transmit']=getdataprometheus(pre_url+expr_network_recieve)

In [16]:
#Pod Stats
expr_pod_cpu_aeneas_function = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn",pod=~"scaling-aeneas-mqtt.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_broker = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="default",pod=~"mosquitto.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_aeneas = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"aeneas.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_mosquitto_connector_tocloud = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas",pod=~"tocloud.*"}['+str(interval)+'s]))&time='+str(time)
expr_pod_cpu_functions_all = 'sum(rate(container_cpu_usage_seconds_total{container_name!="POD",namespace="openfaas-fn"}['+str(interval)+'s]))&time='+str(time)


metrics['pod_cpu_aeneas_function']=getdataprometheus(pre_url+expr_pod_cpu_aeneas_function)
metrics['pod_cpu_mosquitto_broker']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_broker)
metrics['pod_cpu_mosquitto_connector_aeneas']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_aeneas)
metrics['pod_cpu_mosquitto_connector_tocloud']=getdataprometheus(pre_url+expr_pod_cpu_mosquitto_connector_tocloud)
metrics['pod_cpu_functions_all']=getdataprometheus(pre_url+expr_pod_cpu_functions_all)



In [17]:
df_metrics = pd.read_csv("metrics.csv")
df_metrics = df_metrics.append(metrics, ignore_index=True)


/tmp/ipykernel_6005/39894093.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_metrics = df_metrics.append(metrics, ignore_index=True)


In [19]:
df_metrics.to_csv(str(user)+"_"+scenario+"_"+"metrics.csv")
display(df_metrics)

,Unnamed: 0,workload_type,workload_size,Workload TPT,Average Workload TPT,Average FET,Average CT,C_CPU,C_RAM,C_disk_read,C_disk_write,C_network_recieve,C_network_transmit,pod_cpu_aeneas_function,pod_cpu_functions_all,pod_cpu_mosquitto_broker,pod_cpu_mosquitto_connector_aeneas,pod_cpu_mosquitto_connector_tocloud
0,0.0,15_WITHOUT_CONF,15,578,38.533333,4.668633,33.8647,21.5,35.74691,2609.303704,306744.888889,0,577.365476,0.228407,0.239458,0.002038,0.055114,0.027421
1,NaN,10_WITHOUT_CONF,10,291,29.100000,4.687400,24.4126,19.80000000000777,35.52972853480743,19539.437037037038,417829.92592592584,0,586.3777777777779,0.28895965984920824,0.30115251707498014,0.00219679269445128,0.09149128300202528,0.02703537840569712
